<h1 dir=rtl align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
چی بخریم؟
</font>
</h1>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
به یک پروژه‌ی متفاوت و کاربردی خوش آمدید! 🤩 در این مسئله گریزی به <i>سیستم‌های پیشنهاددهنده (Recommender Systems)</i> خواهیم زد و با استفاده از یک معماری یادگیری عمیق خلاقانه، مدلی پیاده خواهیم کرد که بتواند علاقه‌ی یک کاربر به یک کالای مشخص را تخمین بزند. با استفاده از این مدل خواهیم توانست لیستی از کالاها را به کاربر پیشنهاد دهیم که تاکنون آن‌ها را خریداری نکرده اما طبق الگوهایی که مدل بین کاربران و کالاها پیدا کرده به‌احتمال زیاد از آن‌ها خوشش خواهد آمد و به آن‌ها امتیاز بالایی خواهد داد. مجموعه‌داده‌ی پروژه نیز شامل داده‌های واقعی مربوط به کالاهای الکترونیکی موجود در <i>فروشگاه آنلاین آمازون (Amazon)</i> است.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font color="orange"><b>نکته:</b></font>
<font face="vazir" size=3>
اگرچه در این پروژه با داده‌های فروشگاه آنلاین سروکار داریم اما این مدل پیشنهاددهنده را می‌توان در دامنه‌های گوناگونی به کار گرفت. به‌عنوان مثال می‌توان سیستمی مشابه جهت پیشنهاد فیلم یا قطعه‌ی موسیقی به کاربران طراحی کرد.
</font>
</p>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مجموعه‌داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
داده‌های آموزشی در فایل <code dir=ltr>amazon_train.csv</code> در دسترس شما قرار گرفته است. این فایل شامل ۴ ستون است که به‌ترتیب بیانگر شناسه‌ی کاربر، شناسه‌ی محصول، امتیازی که کاربر به محصول داده و زمان ثبت امتیاز هستند. شبکه‌ای که در ادامه پیاده خواهیم کرد شناسه‌ی عددی کاربر و محصول را ورودی می‌گرد و امتیازی که کاربر به آن محصول می‌دهد را تخمین خواهد زد. ابتدا فایل مجموعه‌داده را به‌کمک <code>pandas</code> بخوانید.
</font>
</p>

In [2]:
import numpy as np
import pandas as pd



In [3]:
# Loading the dataset
# TODO

train = pd.read_csv("data/amazon_train.csv")
valid = pd.read_csv('data/amazon_valid.csv')
test = pd.read_csv('data/amazon_test.csv')
train.head()

,UserID,ProductID,Rating,Timestamp
0,A3HICVLF4PFFMN,0594481813,5.0,2014-05-05
1,A2QBZA4S1ROX9Q,0594481813,3.0,2013-05-25
2,AT09WGFUM934H,0594481813,3.0,2013-08-31
3,AGAKHE014LQFU,0594481813,3.0,2013-09-18
4,A1S6B5QFWGVL5U,0594481813,4.0,2013-06-27


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌پردازش
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
پس از بررسی و تحلیل داده‌هایی که در اختیارتان گرفته ممکن است به پیش‌پردازش‌های مختلفی نیاز پیدا کنید و ایده‌های گوناگونی را پیاده کنید. با این حال، دو پیش‌پردازش اصلی که حتماً به آن‌ها نیاز خواهید داشت را در ادامه شرح داده‌ایم.
</font>
</p>

<h3 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
کدگذاری
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
از آن‌جا که شناسه‌های کاربر (<code dir=ltr>UserID</code>) و محصول (<code dir=ltr>ProductID</code>) به‌صورت رشته‌ای هستند، نیاز است آن‌ها را به‌شکل اعداد مرتب کدگذاری کنیم. بنابراین ابتدا با استفاده از کلاس <code>LabelEncoder</code> موجود در کتابخانه‌ی <code>sklearn</code> هر کدام از این ستون‌ها را کدگذاری کنید. پیشنهاد می‌شود حاصل را در ستون‌های مجزایی به‌نام‌های <code>UserEnc</code> و <code>ProductEnc</code> ذخیره کنید.
</font>
</p>

In [4]:
from imblearn.over_sampling import RandomOverSampler

sampling_strategy = {1: round(train['Rating'].value_counts()[1]*2.5),
                     2: round(train['Rating'].value_counts()[2]*3),
                     3: round(train['Rating'].value_counts()[3]*2)}

ros = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(train[['UserID', 'ProductID']], train['Rating'])

train_ros = pd.DataFrame(X_train_ros, columns=['UserID', 'ProductID'])
train_ros['Rating'] = y_train_ros

train_ros.head()

,UserID,ProductID,Rating
0,A3HICVLF4PFFMN,0594481813,5.0
1,A2QBZA4S1ROX9Q,0594481813,3.0
2,AT09WGFUM934H,0594481813,3.0
3,AGAKHE014LQFU,0594481813,3.0
4,A1S6B5QFWGVL5U,0594481813,4.0


In [5]:
train = train_ros

from sklearn.preprocessing import LabelEncoder

le_user = LabelEncoder()
train['UserEnc'] = le_user.fit_transform(train['UserID'].values)

le_product = LabelEncoder()
train['ProductEnc'] = le_product.fit_transform(train['ProductID'].values)

valid['UserEnc'] = le_user.transform(valid['UserID'].values)
valid['ProductEnc'] = le_product.transform(valid['ProductID'].values)

test['UserEnc'] = le_user.transform(test['UserID'].values)
test['ProductEnc'] = le_product.transform(test['ProductID'].values)

<h3 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
نرمال‌سازی
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
اکنون مقادیر ستون <code>Rating</code> را به‌کمک <code>MinMaxScaler</code> کتابخانه‌ی <code>sklearn</code> نرمال کنید تا مقادیر آن به بازه‌ی ۰ تا ۱ نگاشت شود.
</font>
</p>

In [6]:
from sklearn.preprocessing import MinMaxScaler

# Scaling the Rating column
# TODO

scaler = MinMaxScaler()
train['Rating'] = scaler.fit_transform(train['Rating'].values.reshape(-1, 1))
valid['Rating'] = scaler.transform(valid['Rating'].values.reshape(-1, 1))

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مدل‌سازی
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
حال نوبت به مدل‌سازی می‌رسد. ساختار پیشنهادی ما به این شکل است که شبکه دو ورودی خواهد داشت: شناسه‌ی کاربر و شناسه‌ی کالا. برای هر کدام از این شناسه‌ها یک لایه‌ی تعبیه‌ساز (<code>Embedding</code>) جداگانه در نظر می‌گیریم تا به‌ازای هر کاربر یا کالا، یک بردار تعبیه‌ی بامعنا ساخته شود. خروجی هر لایه‌ی تعبیه به‌کمک یک لایه‌ی <code>Flatten</code> مسطح خواهد شد. پس از این مراحل، دو بردار بازنمایی کالا و کاربر در اختیار شما قرار می‌گیرد و می‌توانید پس از آن معماری دلخواه خود را پیاده کنید. به‌عنوان مثال می‌توانید بردار تعبیه‌ی کالا را در بردار تعبیه‌ی کاربر ضرب نقطه‌ای کنید (به‌کمک لایه‌ی <code>dot</code>) و حاصل را از تابع فعال‌ساز <code>sigmoid</code> عبور دهید تا خروجی شبکه یعنی امتیاز کاربر به آن کالا بین ۰ تا ۱ تولید شود. بنابراین ابتدا این ساختار را به‌کمک مدل تابعی (Functional) کراس پیاده‌سازی کنید.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font color="orange"><b>نکته:</b></font>
<font face="vazir" size=3>
دست شما کاملاً برای ایجاد تغییرات در این ساختار باز است. به‌عنوان مثال می‌توانید علاوه بر تولید بردارهای تعبیه، به‌کمک لایه‌های اضافه‌ی <code>Embedding</code> (با اندازه‌ی بردار ۱) یک مقدار بایاس هم برای هر کاربر یا کالا تولید کنید. در نهایت می‌توانید بایاس کاربر، بایاس کالا و حاصل ضرب نقطه‌ای را به‌کمک یک لایه‌ی <code>add</code> جمع کنید. به‌عنوان ایده‌ای دیگر می‌توانید به‌جای آن‌که دو بردار تعبیه را در یکدیگر ضرب کنید از عملیات‌های دیگری همچون الحاق (<code>Concatenate</code>) استفاده کنید. همچنین می‌توانید بر تعداد لایه‌ها بی‌افزایید، یا روش مقداردهی اولیه‌ی وزن‌ها یا رگولاریزیشن هر لایه را تنظیم کنید.
</font>
</p>

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense



# Build the model using the Functional API
num_users = train['UserEnc'].nunique()
num_products = train['ProductEnc'].nunique()
embedding_size = 200

# User embedding input and layers
user_input = Input(shape=(1,), name='user_input')
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_size, name='user_embedding')(user_input)
user_vector = Flatten(name='user_vector')(user_embedding)

# Product embedding input and layers
product_input = Input(shape=(1,), name='product_input')
product_embedding = Embedding(input_dim=num_products, output_dim=embedding_size, name='product_embedding')(product_input)
product_vector = Flatten(name='product_vector')(product_embedding)

# Dot product of user and product vectors
dot_product = Dot(axes=1, name='dot_product')([user_vector, product_vector])

# Output layer with sigmoid activation
output = Dense(1, activation='sigmoid', name='output')(dot_product)

# Create the final model
model = Model(inputs=[user_input, product_input], outputs=output, name='user_product_model')



<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
پس از طراحی معماری شبکه، می‌توانید آن را با ورودی دادن یک لیست شامل دو لیست شناسه‌ی کاربر و شناسه‌ی کالا آموزش دهید. متغیر هدف نیز همان ستون <code>Rating</code> خواهد بود. همچنین در صورتی‌که در لایه‌ی آخر از تابع فعال‌ساز <code>sigmoid</code> برای نورون خروجی استفاده کرده باشید می‌توانید از تابع زیان <code>binary_crossentropy</code> بهره ببرید. در صورت علاقه و نیاز می‌توانید از هر تکنیک دلخواهی در فرآیند آموزش مدل خود استفاده کنید.
</font>
</p>

In [8]:
# Training phase
# TODO
# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Model: "user_product_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ product_input (InputLayer)    │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ user_embedding (Embedding)    │ (None, 1, 200)            │      18,010,000 │ user_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ product_embedding (Embedding) │ (None, 1, 200)            │       9,753,800 │ product_input[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ user_vector (Flatten)         │ (None, 200)               │               0 │ user_embedding[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ product_vector (Flatten)      │ (None, 200)               │               0 │ product_embedding[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dot_product (Dot)             │ (None, 1)                 │               0 │ user_vector[0][0],         │
│                               │                           │                 │ product_vector[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output (Dense)                │ (None, 1)                 │               2 │ dot_product[0][0]          │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 27,763,802 (105.91 MB)

 Trainable params: 27,763,802 (105.91 MB)

 Non-trainable params: 0 (0.00 B)

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌بینی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
دو فایل دیگر به نام‌های <code>amazon_valid.csv</code> و <code>amazon_test.csv</code> نیز به‌عنوان مجموعه‌ی اعتبارسنجی و آزمون در اختیار شما قرار گرفته است. مدل شما باید بتواند به‌ازای هر جفت کاربر و کالا، امتیازی که آن کاربر به آن کالا می‌دهد را به‌درستی تخمین بزند. ساختار این مجموعه‌ها نیز مشابه با مجموعه‌ی آموزشی است با این تفاوت که برچسب حقیقی داده‌های آزمون در دسترس شما نیست و ارزیابی مدل شما بر روی داده‌های آزمون در سامانه‌ی داوری کوئرا انجام می‌پذیرد. با این حال، توزیع داده‌ها بین مجموعه‌ی اعتبارسنجی و آزمون بسیار مشابه است و اگر مدل شما برای نمونه‌های اعتبارسنجی به عملکرد قابل قبولی برسد به احتمال زیاد برای نمونه‌های آزمون نیز همین‌گونه خواهد بود.
</font>
</p>


<h3 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
هدفی که در این نوع مسئله در نظر گرفته‌ایم این است که مدل شما تا چه‌اندازه می‌تواند کالاهایی که کاربر به آن‌ها امتیاز بالا (۴ یا ۵) می‌دهد را به‌درستی تشخیص دهد، بنابراین
عملکرد مدل شما در سامانه‌ی داوری به این‌گونه ارزیابی خواهد شد که برچسب‌های <code>5</code> و <code>4</code> به‌عنوان دسته‌ی مثبت و سایر برچسب‌ها به‌عنوان دسته‌ی منفی در نظر گرفته می‌شود.
سپس از معیار <code>Precision</code> جهت ارزیابی مدل شما استفاده خواهد شد که در واقع نسبت تعداد پیشنهادات مثبت درست (مدل تشخیص داده که کاربر به کالا امتیاز بالا می‌دهد و واقعاً این‌گونه بوده) به کل پیشنهادات مثبت مدل را می‌شمارد.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font color="red"><b color='red'>توجه:</b></font>
<font face="vazir" size=3>
 جهت کسب امتیاز کامل نیاز است تا پاسخ شما حداقل امتیاز <code>80</code> را با توجه به معیار معرفی‌شده کسب نماید.
</font>
</p>

In [9]:
user_ids = train['UserEnc'].values
product_ids = train['ProductEnc'].values
ratings = train['Rating'].values


valid_user_ids = valid['UserEnc'].values
valid_product_ids = valid['ProductEnc'].values
valid_ratings = valid['Rating'].values

In [10]:
history = model.fit(
    [user_ids, product_ids],  
    ratings,
    validation_data=([valid_user_ids, valid_product_ids], valid_ratings),  
    epochs=10,
    batch_size=1024
)

Epoch 1/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 341s 301ms/step - accuracy: 0.4719 - loss: 0.6473 - val_accuracy: 0.6138 - val_loss: 0.5144
Epoch 2/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 375s 295ms/step - accuracy: 0.5791 - loss: 0.3749 - val_accuracy: 0.6123 - val_loss: 0.4853
Epoch 3/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 401s 357ms/step - accuracy: 0.5871 - loss: 0.2781 - val_accuracy: 0.6121 - val_loss: 0.4864
Epoch 4/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 493s 402ms/step - accuracy: 0.5866 - loss: 0.2786 - val_accuracy: 0.6122 - val_loss: 0.4869
Epoch 5/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 409s 364ms/step - accuracy: 0.5861 - loss: 0.2764 - val_accuracy: 0.6129 - val_loss: 0.4870
Epoch 6/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 354s 315ms/step - accuracy: 0.5875 - loss: 0.2696 - val_accuracy: 0.6124 - val_loss: 0.4878
Epoch 7/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 402s 358ms/step - accuracy: 0.5870 - loss: 0.2700 - val_accuracy: 0.6128 - val_loss: 0.4883
Epoch 8/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 441s 393ms/step - ac

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
ساختار خروجی برای نمونه‌های آزمون
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
 پیش‌بینی مدل خود را برای نمونه‌های آزمون به‌شکل امتیازی بین ۱ تا ۵ تولید کرده و در ستون <code>Rating</code> یک دیتافریم با نام <code>submission</code> در قالب جدول زیر ذخیره کنید.
</font>
</p>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|<code>UserID</code>|شناسه‌ی کاربر (مشابه با نمونه‌ی آزمون)|
|<code>ProductID</code>|شناسه‌ی کالا (مشابه با نمونه‌ی آزمون)|
|<code>Rating</code>|امتیاز کاربر به کالا از ۱ تا ۵ (پیش‌بینی مدل)|
    
</font>
</div>
</center>

In [11]:
user_values_test = test['UserEnc'].values
product_values_test = test['ProductEnc'].values

ID = test['ProductID']
User = test['UserID']

In [14]:
# Preparing submission file
Prediction = model.predict([user_values_test,product_values_test])
rescaled_Prediction = scaler.inverse_transform(Prediction)
if rescaled_Prediction.ndim > 1:
    rescaled_Prediction = rescaled_Prediction.reshape(-1)

# Check shapes of all components to debug
print(f"Shape of User: {User.shape}")
print(f"Shape of ID: {ID.shape}")
print(f"Shape of rescaled_Prediction: {rescaled_Prediction.shape}")

predictions = np.round(rescaled_Prediction)

submission = pd.DataFrame({'UserID':User,'ProductID':ID,'Rating':predictions})
submission

2805/2805 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
Shape of User: (89732,)
Shape of ID: (89732,)
Shape of rescaled_Prediction: (89732,)


,UserID,ProductID,Rating
0,A2Y3A341VDK37H,B00HFI55N2,4.0
1,A240FRPD4MEXND,B00KIMX4EY,5.0
2,A3SBTW3WS4IQSN,B001IF252M,4.0
3,A1QCZQTJNK3MU,B00J46VVKE,4.0
4,A16JT7E5121OAB,B00JZC972Q,4.0
...,...,...,...
89727,A37Z65SZVT0TVB,B00005AR4L,4.0
89728,A3PLX6PTM2ERKL,B00004Z6PI,5.0
89729,A3TRPVAGT3NWBS,B00005NVPW,4.0
89730,AY8Q1X7G96HV5,B000059MSI,4.0


In [19]:
submission['Rating'].value_counts()

4.0    48445
5.0    39699
3.0     1491
2.0       95
1.0        2
Name: Rating, dtype: int64

In [16]:
from sklearn.metrics import precision_score

predictions = model.predict([valid_user_ids, valid_product_ids])
predictions = scaler.inverse_transform(predictions)
predictions = np.round(predictions)
predictions[predictions < 4] = 0
predictions[predictions >= 4] = 1

true_rating = valid['Rating'].values.copy()
true_rating[true_rating < 4] = 0
true_rating[true_rating >= 4] = 1

result = precision_score(true_rating, predictions) * 100
print(result)

2810/2810 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step
0.0


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) تا در صورت نیاز به پشتیبانی امکان بررسی کد شما وجود داشته باشد. همچنین اگر از گوگل کولب استفاده می‌کنید، در صورت نیاز به پشتیبانی حتماً آخرین نسخه از نت‌بوک را به‌صورت دستی دانلود کرده و داخل فایل ارسالی قرار دهید یا لینک کولب را با ما به‌اشتراک بگذارید.
</font>
</p>

In [17]:
import zipfile
import joblib
import os

if not os.path.exists(os.path.join(os.getcwd(), 'what_to_buy.ipynb')):
    %notebook -e what_to_buy.ipynb

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)
file_names = ['what_to_buy.ipynb', 'submission.csv']
compress(file_names)

File Paths:
['what_to_buy.ipynb', 'submission.csv']


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
بیشتر بدانید:
سیستم پیشنهاددهنده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
برای پیشنهاد کالای جدید به یک کاربر روش‌های مختلفی وجود دارد، اما یک راه ساده این است که از تخمین امتیازی که مدل به‌ازای هر جفت کاربر و کالا می‌دهد استفاده کنیم.
به‌ازای یک کاربر به‌کمک مدل خود تخمین می‌زنیم که وی از بین کالاهایی که تاکنون خریداری نکرده، کدام‌ها را بیشتر از همه دوست خواهد داشت و امتیاز بالاتری به آن‌ها خواهد داد. بنابراین می‌توانیم این کالاها را به او پیشنهاد دهیم و به‌احتمال زیاد از تعدادی از آن‌ها خوشش خواهد آمد. البته این موضوع نمی‌تواند مستقیماً به این معنی باشد که آن کالا را خریداری می‌کند. به‌عنوان مثال شاید یک کالای گران‌قیمت را پیشنهاد دهیم که همگی از آن خوشمان می‌آید اما بسیاری قدرت خرید آن را نداریم! 🫠
</font>
</p>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
<span style="color:green"><b>راهنمایي:</b></span>
برای تولید پیش‌بینی به‌ازای یک کاربر ابتدا لیستی از کالاهایی که وی تاکنون خریداری نکرده را پیدا کنید. سپس جفت شناسه‌ی کدگذاری‌شده‌ی کاربر و هر یک از این کالاها را به مدل ورودی بدهید تا امتیاز کاربر به هر کدام از آن کالاها تخمین زده شود. در نهایت می‌توان <code>n</code> کالایی که منجر به بیشترین امتیاز (خروجی مدل) شده را انتخاب کرد و به وی پیشنهاد داد.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
<span style="color:purple"><b>🧐 بیشتر بی‌اندیشیم:</b></span>
چه روش‌های دیگری جهت پیشنهاد کالا به یک کاربر مشخص به ذهن‌تان می‌رسد؟ آیا می‌توان مستقیماً‌ از بردارهای تعبیه‌ی هر کالا یا کاربر جهت پیشنهاد استفاده کرد؟ به چه روش‌های می‌توان کاربران مشابه با یک کاربر را پیدا کرده و براساس تجربه یا الگوی خرید آن‌ها به کاربر کالاهایی را پیشنهاد داد؟ پیشنهاد می‌کنیم به این موضوعات بیشتر فکر کرده و نظرات خود را از طریق بخش دیدگاه عمومی صفحه‌ی پروژه در کوئرا با دیگران به‌اشتراک بگذارید. چه بهتر که ایده‌های خود را به‌شکل عملی نیز آزمایش کنید.
</font>
</p>